In [1]:
from MixedAutoencoder import Mixer, MixedAutoencoder
import MixedAutoencoder
from DataCleaning import *
import DataCleaning
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
seed = 42
MixedAutoencoder.setRandom(seed)
DataCleaning.setRandom(seed)
base_path = "."
mixer = Mixer(base_path)

In [2]:

num_sets = 4
latent_dim = 8
model_shape = []
base_path = "."
label = f'demo_{num_sets}_{latent_dim}_[{"_".join([str(s) for s in model_shape])}]'
demo_size = 15
demo_shape = []


In [3]:
base_key_list = [f'set{str(i)}' for i in range(1, num_sets+1)]
demo_key_list = ['demo']
data = pd.read_csv(f'{base_path}/data/16PF/data.csv', sep="\t")
data = clean_data(data)
data = data.sample(frac=1)
demo_column_keys = {demo_key_list[0]: split_n_strat(data, demo_size)}

base_column_keys = split_strat([k for k in list(data) if k not in demo_column_keys[demo_key_list[0]]],num_sets, base_key_list)
split_data = split(data, base_column_keys)
train, test = make_train_test(split_data, 0.8)
input_dims = get_input_dims(train)

In [4]:
model_shapes = {k: model_shape for k in base_key_list}
autoencoder_set = mixer.make_new(model_shapes, latent_dim, input_dims)

autoencoder_set.show_total_binary_accuracy(test)
settings = {
    "training": [["$all", "$all", True, True]]#[[[k1], [k2 for k2 in keys if k1 != k2], True, True] for k1 in keys],
    #"encoder_proximity_training": [["$all", True]],
    #"plot": [True, 3, [0, 1, 2]]
}
autoencoder_set.train_set(train, 25, autoencoder_set.make_train_config(settings = settings), batch_size = 64, verbose=True)
autoencoder_set.show_total_binary_accuracy(test)

Binary accuracy: 0.49523964861434355
epoch 0
442/442 [==============================] - 6s 11ms/step - loss: 1.2121 - val_loss: 1.1430 - accuracy: 0.3015
epoch 1
442/442 [==============================] - 4s 6ms/step - loss: 1.0394 - val_loss: 0.9640 - accuracy: 0.3762
epoch 2
442/442 [==============================] - 5s 6ms/step - loss: 0.9759 - val_loss: 0.8815 - accuracy: 0.4049
epoch 3
442/442 [==============================] - 4s 6ms/step - loss: 0.9396 - val_loss: 0.8163 - accuracy: 0.4209
epoch 4
442/442 [==============================] - 5s 6ms/step - loss: 0.9163 - val_loss: 0.7801 - accuracy: 0.4358
epoch 5
442/442 [==============================] - 5s 7ms/step - loss: 0.8985 - val_loss: 0.7601 - accuracy: 0.4440
epoch 6
442/442 [==============================] - 5s 6ms/step - loss: 0.8839 - val_loss: 0.7495 - accuracy: 0.4510
epoch 7
442/442 [==============================] - 5s 6ms/step - loss: 0.8727 - val_loss: 0.7431 - accuracy: 0.4541
epoch 8
442/442 [=================

In [5]:
model_shapes_3d = {k: [8] for k in base_key_list}
autoencoder_set_3d = mixer.make_new(model_shapes_3d, 3, input_dims)
autoencoder_set_3d.train_set(train, 25, autoencoder_set_3d.make_train_config(settings = settings), batch_size = 64, verbose=False)
autoencoder_set_3d.show_total_binary_accuracy(test)

Binary accuracy: 0.7628171912149404


In [6]:
demo_split_data = split(data, demo_column_keys)
dtrain, dtest = make_train_test(demo_split_data, 0.8)
demo_input_dims = get_input_dims(dtrain)




demo_shapes = {k: demo_shape for k in demo_key_list}
autoencoder_set = mixer.add_new(autoencoder_set, demo_shapes, demo_input_dims)

settings = {
    "training": [[demo_key_list, "$all", True, False], ["$all", demo_key_list, False, True]]#[[[k1], [k2 for k2 in keys if k1 != k2], True, True] for k1 in keys],
    #"encoder_proximity_training": [["$all", True]],
    #"plot": [True, 3, [0, 1, 2]]
}
autoencoder_set.train_set(train | dtrain, 25, autoencoder_set.make_train_config(settings = settings), batch_size = 64, verbose=True)


epoch 0
442/442 [==============================] - 3s 7ms/step - loss: 1.3415 - val_loss: 1.4457 - accuracy: 0.2966
epoch 1
442/442 [==============================] - 3s 4ms/step - loss: 1.0806 - val_loss: 1.1645 - accuracy: 0.3495
epoch 2
442/442 [==============================] - 3s 4ms/step - loss: 0.9363 - val_loss: 0.9831 - accuracy: 0.3990
epoch 3
442/442 [==============================] - 3s 4ms/step - loss: 0.8677 - val_loss: 0.9062 - accuracy: 0.4220
epoch 4
442/442 [==============================] - 3s 4ms/step - loss: 0.8386 - val_loss: 0.8748 - accuracy: 0.4361
epoch 5
442/442 [==============================] - 3s 4ms/step - loss: 0.8258 - val_loss: 0.8614 - accuracy: 0.4341
epoch 6
442/442 [==============================] - 3s 4ms/step - loss: 0.8193 - val_loss: 0.8541 - accuracy: 0.4348
epoch 7
442/442 [==============================] - 3s 5ms/step - loss: 0.8157 - val_loss: 0.8495 - accuracy: 0.4348
epoch 8
442/442 [==============================] - 3s 5ms/step - loss: 0

In [7]:
demo_shapes_3d = {k: [8] for k in demo_key_list}
autoencoder_set_3d = mixer.add_new(autoencoder_set_3d, demo_shapes_3d, demo_input_dims)
autoencoder_set_3d.train_set(train | dtrain, 25, autoencoder_set_3d.make_train_config(settings = settings), batch_size = 64, verbose=False)
autoencoder_set_3d.show_binary_accuracy(demo_key_list, test | dtest)

Encoder binary accuracy: 0.7538527166310137
Decoder binary accuracy: 0.7975246306861448


In [8]:
autoencoder_set.show_binary_accuracy(demo_key_list, test | dtest)

Encoder binary accuracy: 0.7717992308995214
Decoder binary accuracy: 0.8264166305455654


In [9]:

mixer.save_to_label(autoencoder_set, extra = {"columns" : base_column_keys|demo_column_keys}, label = label)

INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]/encoder_set1\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]/decoder_set1\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]/encoder_set2\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]/decoder_set2\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]/encoder_set3\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]/decoder_set3\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]/encoder_set4\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]/decoder_set4\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]/encoder_demo\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]/decoder_demo\assets


In [10]:
mixer.save_to_label(autoencoder_set_3d, extra = {"columns" : base_column_keys|demo_column_keys}, label = label+"_3d")

INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]_3d/encoder_set1\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]_3d/decoder_set1\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]_3d/encoder_set2\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]_3d/decoder_set2\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]_3d/encoder_set3\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]_3d/decoder_set3\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]_3d/encoder_set4\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]_3d/decoder_set4\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]_3d/encoder_demo\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_8_[]_3d/decoder_demo\assets
